<a href="https://colab.research.google.com/github/Fmarcelneves/exemplos-de-uso-do-ngrok-no-google-colab/blob/main/Exemplo_do_ngrok_com_streamlit_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import and install packages 

In [ ]:
%%capture
!pip install streamlit
!pip install pyngrok==4.1.1
from pyngrok import ngrok

## Streamlit app example

In [ ]:
%%writefile app.py
import numpy as np
import pandas as pd 
import streamlit as st

st.title('Streamlit example - Uber pickups in NYC')

DATE_COLUMN = 'date/time'
DATA_URL = ('https://s3-us-west-2.amazonaws.com/'
            'streamlit-demo-data/uber-raw-data-sep14.csv.gz')

@st.cache
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows)
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[DATE_COLUMN] = pd.to_datetime(data[DATE_COLUMN])
    return data

data_load_state = st.text('Loading data...')
data = load_data(10000)
data_load_state.text("Done! (using st.cache)")

if st.checkbox('Show raw data'):
    st.subheader('Raw data')
    st.write(data)

st.subheader('Number of pickups by hour')
hist_values = np.histogram(data[DATE_COLUMN].dt.hour, bins=24, range=(0,24))[0]
st.bar_chart(hist_values)

# Some number in the range 0-23
hour_to_filter = st.slider('hour', 0, 23, 17)
filtered_data = data[data[DATE_COLUMN].dt.hour == hour_to_filter]

st.subheader('Map of all pickups at %s:00' % hour_to_filter)
st.map(filtered_data)

Overwriting app.py


In [ ]:
!streamlit run --server.port 8050 app.py &>/dev/null&

## ngrok

In [ ]:
! ngrok authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Open a HTTP tunnel on the port 8050
public_url = ngrok.connect('8050')

In [ ]:
public_url

'http://11bf30f0bb5c.ngrok.io'

In [ ]:
#use ngrok kill to kill the URL
#ngrok.kill()